# Data Importing

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 

In [ ]:
#import data
parking_data = pd.read_csv("../data/Parking_Violations_Issued_-_Fiscal_Year_2020.csv")
parking_data.head()

In [ ]:
# pick which columns are we using
parking_df = parking_data.copy()
new_columns = ['Summons Number', 'Plate ID', 'Registration State', 'Plate Type',
       'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make',
       'Issuing Agency', 'Violation Location',
       'Violation Time', 'Violation County',
       'Vehicle Color']
parking_df = parking_df.drop([c for c in parking_df.columns if c not in new_columns], axis='columns')
parking_df.head()

In [ ]:
print(f'Number of Rows: {len(parking_df)}') # 12m

In [ ]:
parking_df.info()

In [ ]:
parking_df.isnull().sum()

# Data Cleaning

### Drop missing data

In [ ]:
parking_df = parking_df.dropna()
print(f"Number of Rows: {len(parking_df)}") #7m data

### Filter some of the data
- Registration Plate: some data are in strings of number ( Example: "99")
- Plate Type: some data are in strings of numbers( Example : "999" )
- Violation Code: need to be in range of 1 to 99 

In [ ]:
parking_df = parking_df[(parking_df['Registration State'] != "99") & (parking_df['Plate Type'] != "999")
                 & (parking_df['Violation Code'] != 0) & (parking_df['Violation Code'] < 99) & (parking_df['Vehicle Make'].notnull())
                 & (parking_df['Violation Time'].notnull())]

print(f'Number of Rows: {len(parking_df)}')

### Drop duplicates


In [ ]:
parking_df = parking_df.drop_duplicates()
print(f'Number of Rows: {len(parking_df)}')

### Issue Date(Column) formate to datetime object

In [ ]:
parking_df['Issue Date']= pd.to_datetime(parking_df['Issue Date'])

In [ ]:
parking_df.head()

### Change Violation Time to datetime object.
1. Make sure that every rows has the formate of HHMMA/P
2. Add M to the end of each row 
3. Convert to datetime

In [ ]:
# clean up the time that does not match the pattern
print(f"Before: {len(parking_df)}")
parking_df = parking_df[parking_df['Violation Time'].str.match(pat = '^[0-1][0-9][0-5][0-9][A|P]$')]
print(f"After: {len(parking_df)}")

In [ ]:
from datetime import datetime

def string_date_format(violation_time):
    violation_hr = float(violation_time[0:2])
    violation_min = float(violation_time[2:4])
    violation_frame = violation_time[4:5]
    violation_time = violation_time+"M"
    if(violation_hr >= 0 and violation_hr <= 12 and (violation_frame == "P" or violation_frame == "A")):
        if(violation_time[0:2] == "00" ):
            violation_time = "12"+violation_time[2:]
        violation_time = datetime.strptime(violation_time, '%I%M%p')
    else:
        violation_time =pd.NaT
    return violation_time

In [ ]:
parking_df['Violation Time'] = parking_df['Violation Time'].apply(string_date_format)

### Spliting Data into 2020 and 2019

In [ ]:
import datetime 

parking_df_2020 = parking_df.loc[(parking_df['Issue Date'] >= '2020-01-01') & (parking_df['Issue Date'] <=  '2020-12-31')]
parking_df_2019 = parking_df.loc[(parking_df['Issue Date'] >= '2019-01-01') & (parking_df['Issue Date'] <= '2019-12-31')]

In [ ]:
print(f"2020: {len(parking_df_2020)}")
print(f"2019: {len(parking_df_2019)}")

# Data Exporting ( Clean Data )
Exporting all the clean data so we can do analyst in another notebook

In [ ]:
# parking_df_2020.to_csv('../data/parking_df_2020.csv')
# parking_df_2019.to_csv('../data/parking_df_2019.csv')
parking_df.to_csv('../data/clean_parking_df.csv')